In [ ]:
import numpy as np
import healpy as hp
import bokeh
import colorcet as cc
import pandas as pd
import panel as pn
from astropy.time import Time
import astropy.coordinates

import healsparse as hsp

import schedview.collect.scheduler_pickle
from schedview.plot.spheremap import (
    Planisphere,
    ArmillarySphere,
    MollweideMap,
    HorizonMap,
    split_healpix_by_resolution,
)
from schedview.compute.camera import LsstCameraFootprintPerimeter
from rubin_sim.scheduler.model_observatory.model_observatory import ModelObservatory
import schedview.compute.astro
from schedview.collect.stars import load_bright_stars
from rubin_sim.scheduler.utils.footprints import get_dustmap

from rubin_sim.scheduler.utils.sky_area import SkyAreaGenerator

In [ ]:
pn.extension()

In [ ]:
%%html
<script src="https://unpkg.com/gpu.js@latest/dist/gpu-browser.min.js"></script>

In [ ]:
BAND_COLORS = dict(
    u="#56b4e9", g="#008060", r="#ff4000", i="#850000", z="#6600cc", y="#222222"
)
BAND_HATCH_PATTERNS = dict(
    u="dot",
    g="ring",
    r="horizontal_line",
    i="vertical_line",
    z="right_diagonal_line",
    y="left_diagonal_line",
)
BAND_HATCH_SCALES = dict(u=6, g=6, r=6, i=6, z=12, y=12)

NSIDE_LOW = 8

In [ ]:
observatory = ModelObservatory()
night = Time('2025-03-01', location=observatory.location)

# Simple planisphere example

In [ ]:
plot = bokeh.plotting.figure(
    plot_width=512,
    plot_height=512,
    match_aspect=True,
    title="Sample 1",
)
psphere = Planisphere(mjd=night.mjd, plot=plot)
psphere.add_mjd_slider()
psphere.add_graticules(graticule_kwargs={'min_decl': -80, 'max_decl': 80, 'decl_space': 20, 'min_ra': 0, 'max_ra': 360, 'ra_space': 30}, line_kwargs={'color': 'lightgray'})
psphere.add_ecliptic(color='green')
psphere.add_galactic_plane(color='blue')
psphere.add_horizon()
psphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})
pn.Row(psphere.figure)

# Simple armillary sphere example

In [ ]:
plot = bokeh.plotting.figure(
    plot_width=512,
    plot_height=512,
    match_aspect=True,
    title="Sample 2",
)
asphere = ArmillarySphere(mjd=night.mjd, plot=plot)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon()
asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})
pn.Row(asphere.figure)

# Multiple connected views

In [ ]:
data_source = {}

asphere_plot = bokeh.plotting.figure(
    plot_width=512,
    plot_height=512,
    match_aspect=True,
    title="Sample 3a",
)
asphere = ArmillarySphere(mjd=night.mjd, plot=asphere_plot)
mjd_slider = asphere.add_mjd_slider()

asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
data_source['horizon'] = asphere.add_horizon()
data_source['high_X'] = asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})

psphere_plot = bokeh.plotting.figure(
    plot_width=512,
    plot_height=512,
    match_aspect=True,
    title="Sample 3b",
)
psphere = Planisphere(mjd=night.mjd, plot=psphere_plot)
psphere.add_graticules(graticule_kwargs={'min_decl': -80, 'max_decl': 80, 'decl_space': 20, 'min_ra': 0, 'max_ra': 360, 'ra_space': 30}, line_kwargs={'color': 'lightgray'})
psphere.add_ecliptic()
psphere.add_galactic_plane()
psphere.add_horizon(data_source=data_source['horizon'])
psphere.add_horizon(data_source=data_source['high_X'], line_kwargs={"color": "red", "line_width": 2})

msphere_plot = bokeh.plotting.figure(
    plot_width=1024,
    plot_height=512,
    match_aspect=False,
    title="Sample 3c",
)
msphere = MollweideMap(mjd=night.mjd, plot=msphere_plot)

msphere.add_graticules(graticule_kwargs={'min_decl': -80, 'max_decl': 80, 'decl_space': 20, 'min_ra': 0, 'max_ra': 360, 'ra_space': 30}, line_kwargs={'color': 'lightgray'})
# HACK to make the RA=180 graticule appear both on the left and right
msphere.add_graticules(graticule_kwargs={'min_decl': -80, 'max_decl': 80, 'decl_space': 160, 'min_ra': 180-1e-6, 'max_ra': 180, 'ra_space': 30}, line_kwargs={'color': 'lightgray'})

msphere.add_ecliptic()
msphere.add_galactic_plane()
msphere.add_horizon(data_source=data_source['horizon'])
msphere.add_horizon(data_source=data_source['high_X'], line_kwargs={"color": "red", "line_width": 2})

pn.Column(msphere.figure,
          pn.Row(asphere.figure, psphere.figure))

# Add the sun, moon, and stars

Use `astropy` to get the position of the sun and moon:

In [ ]:
sun_coords = astropy.coordinates.get_sun(night)
moon_coords = astropy.coordinates.get_moon(night)
sun_coords, moon_coords

Load the Yale bright star catalog:

In [ ]:
try:
    stars = load_bright_stars()
except FileNotFoundError:
    stars = load_bright_stars('http://tdc-www.harvard.edu/catalogs/bsc5.dat.gz')
    
stars

This is way too many stars. Only consider the brightest:

In [ ]:
stars.query('Vmag<3.5', inplace=True)

Now show the figure:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon()
asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})

asphere.add_marker(sun_coords.ra.deg, sun_coords.dec.deg, name="Sun", glyph_size=15, circle_kwargs={"color": "brown"})
asphere.add_marker(moon_coords.ra.deg, moon_coords.dec.deg, name="Moon", glyph_size=15, circle_kwargs={"color": "orange"})

# Scale the size of the star markers with the magnitude of the stars
stars["glyph_size"] = 15 * (1.01 - stars['Vmag']/stars['Vmag'].max())

# Actually add the stars
asphere.add_stars(stars, mag_limit_slider=True, star_kwargs={"color": "black"})

# Set the limit of the slider according to the stars we've included
asphere.sliders['mag_limit'].end = stars['Vmag'].max()

pn.Row(asphere.figure)

# Horizon coordinates

You can show a map in a horizon (az/alt polar) projection:

In [ ]:
hsphere = HorizonMap(mjd=night.mjd)
hsphere.add_mjd_slider()
hsphere.add_horizon_graticules()
hsphere.add_horizon()
hsphere.add_ecliptic()
hsphere.add_galactic_plane()

hsphere.add_marker(sun_coords.ra.deg, sun_coords.dec.deg, name="Sun", glyph_size=15, circle_kwargs={"color": "brown"})
hsphere.add_marker(moon_coords.ra.deg, moon_coords.dec.deg, name="Moon", glyph_size=15, circle_kwargs={"color": "orange"})
hsphere.add_stars(stars, mag_limit_slider=True, star_kwargs={"color": "black"})

pn.Row(hsphere.figure)

You can show horizon graticules in any of the projections:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon_graticules(line_kwargs={'color': "red", 'line_dash': 'dotted'})
pn.Row(asphere.figure)

# Show a healpix map

Get the healpix dust map from `rubin_sim` as an example healpix map.

In [ ]:
dust = get_dustmap(nside=32)

Make a `bokeh` color map:

In [ ]:
cmap = bokeh.transform.log_cmap(
        'value', cc.palette['CET_L18'], np.quantile(dust, 0.75), np.quantile(dust, 0.99)
)

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_healpix(dust, nside=32, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color='lightgreen')
asphere.add_galactic_plane(color='lightblue')
pn.Row(asphere.figure)

# Show a healsparse map

At nsides greater than 32, interactivity can be sluggish. Sometimes you can reduce the number of pixels by only displaying some regions of the sky. For example, we can show only the high dust areas in the dust map above:

In [ ]:
# Get a higher resolution healpix map
dust64 = hp.reorder(get_dustmap(nside=64), inp="RING", out="NESTED")

# Cut off any pixels lower than the bottom for our color map
dust64[dust64<cmap['transform'].low] = hp.UNSEEN

# Make a healsparse map with just the seen healpixel
dust_hsp = hsp.HealSparseMap(nside_coverage=16, healpix_map=dust64)

Make a color map such that low dust areas are near white, and thus fall to white as dust drops:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_healpix(dust_hsp, nside=64, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color='lightgreen')
asphere.add_galactic_plane(color='lightblue')
pn.Row(asphere.figure)

# Survey footprint

Get the final target survey footprint:

In [ ]:
nside = 64
sky_area_generator = SkyAreaGenerator(nside=nside)
footprint, footprint_pix_labels = sky_area_generator.return_maps()

Split the desired footprint between edges between regions, which we will show using the full nside healsparse map, and a lower-nside healsparse map on large uniform areas:

In [ ]:
low_nside = 16
this_footprint = footprint['g'].copy()
this_footprint[this_footprint==0] = hp.UNSEEN
footprint_high, footprint_low = split_healpix_by_resolution(
        this_footprint, low_nside, nside
    )

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)

cmap = bokeh.transform.linear_cmap(
        'value', cc.palette['rainbow4'], 0, 1
)

asphere.add_healpix(footprint_high, nside=nside, cmap=cmap)
asphere.add_healpix(footprint_low, nside=low_nside, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color='lightgreen')
asphere.add_galactic_plane(color='lightblue')
pn.Row(asphere.figure)

# Arbitrary bokeh

<span style="color:red; font-size:x-large">A planned refactor of `spheremap` will result in a change in the API used in this section</span>

See PREOPS-3405.

## Data sources with point locations

Use `asphere.make_points` to create the data source.

The `plot` member of `SphereMap` and its children (`ArmillarySphere`, etc.) is just a normal `bokeh.plotting.Figure`.

The `make_points` method of any of these objects generate a `bokeh.models.ColumnDataSource` with columns with the coordinates on the projection plane for the different projections. A client-side javascript callback updates these columns as necessary. The `x_col` and `y_col` members of `SphereMap`'s children hold the names of the columns that have the `x` and `y` coordinates in the projection plane.

So, to plot on the projection plane using any of the varous methods of `bokeh.plotting.Figure` that take single sets of coordinates for each value, create the appropriate data source using `make_points`, and call the appropriate member of

In [ ]:
npoints = 100
sample_df = pd.DataFrame({
    'name': 'sample',
    'glyph_size': 10,
    'ra': np.random.random(npoints)*360,
    'decl': np.random.random(npoints)*180 - 90
})
asphere = ArmillarySphere(mjd=night.mjd)

sample_ds = asphere.make_points(sample_df)
asphere.plot.asterisk(asphere.x_col, asphere.y_col, size='glyph_size', source=sample_ds)
asphere.add_graticules()
asphere.add_ecliptic(color='lightgreen')
asphere.add_galactic_plane(color='lightblue')
pn.Row(asphere.figure)

## Data sources with paths

Use `asphere.make_patches_data_source` to create the data source.

In [ ]:
npoints = 25
sample_df = pd.DataFrame({
    'name': 'sample',
    'glyph_size': 10,
    'center_ra': np.random.random(npoints)*360,
    'center_decl': np.random.random(npoints)*180 - 90,
    'rotation': 180*np.random.random(npoints)
})

camera_perimeter = LsstCameraFootprintPerimeter()
ras, decls = camera_perimeter(
            sample_df.center_ra, sample_df.center_decl, sample_df.rotation
        )
sample_df['ra'] = ras
sample_df['decl'] = decls
sample_df.head()

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
sample_ds = asphere.make_patches_data_source(sample_df)
asphere.plot.patches(xs=asphere.x_col,
            ys=asphere.y_col,
            source=sample_ds,
            line_color='blue',
            fill_color='red',
            fill_alpha=0.2
        )
asphere.add_graticules()
asphere.add_ecliptic(color='lightgreen')
asphere.add_galactic_plane(color='lightblue')
pn.Row(asphere.figure)

The end.